In [5]:
from google.colab import files
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import io

# Função para carregar arquivos
def carregar_arquivo_colab(mensagem):
    uploaded = files.upload()
    nome_arquivo = next(iter(uploaded))
    return pd.read_excel(io.BytesIO(uploaded[nome_arquivo]))

print("Carregue o arquivo de ORDENS:")
ordens = carregar_arquivo_colab("sample.xlsx")

print("\nCarregue o arquivo da MATRIZ DE DISTÂNCIAS:")
matriz_dist = carregar_arquivo_colab("matriz_filtrada.xlsx")

Carregue o arquivo de ORDENS:


StopIteration: 

In [11]:
class Otimizador:
    def __init__(self, num_empilhadeiras):
        self.num_empilhadeiras = num_empilhadeiras
        self.resetar()

    def resetar(self):
        self.empilhadeiras = {
            i: {
                'posicao': None,
                'livre_em': datetime.now(),
                'distancia_total': 0.0,
                'ordens_atendidas': []
            } for i in range(self.num_empilhadeiras)
        }
        self.ordens_nao_atendidas = []

    def otimizar(self, ordens, matriz_dist):
        self.resetar()

        # Pré-processamento
        ordens['data_hora'] = pd.to_datetime(ordens['data_hora'], errors='coerce')
        ordens = ordens.dropna(subset=['data_hora']).sort_values('data_hora')

        matriz_dist = matriz_dist.set_index(matriz_dist.columns[0])
        matriz_dist = matriz_dist.map(lambda x: float(str(x).replace(',', '.')))

        for _, ordem in ordens.iterrows():
            self.processar_ordem(ordem, matriz_dist)

        return self.gerar_resultados(matriz_dist)

    def processar_ordem(self, ordem, matriz_dist):
        melhor_emp = None
        melhor_custo = float('inf')

        for emp_id, emp in self.empilhadeiras.items():
            try:
                pos_atual = emp['posicao'] or ordem['origem']
                dist = matriz_dist.loc[pos_atual, ordem['origem']] + \
                       matriz_dist.loc[ordem['origem'], ordem['destino']]

                tempo_espera = max(0, (ordem['data_hora'] - emp['livre_em']).total_seconds())
                custo = dist + (tempo_espera * 0.1)

                if custo < melhor_custo:
                    melhor_custo = custo
                    melhor_emp = emp_id

            except Exception as e:
                print(f"Erro na ordem {ordem.get('ordem', '?')}: {str(e)}")
                continue

        if melhor_emp is not None:
            self.atribuir_ordem(melhor_emp, ordem, matriz_dist)
        else:
            self.ordens_nao_atendidas.append(ordem.to_dict())

    def atribuir_ordem(self, emp_id, ordem, matriz_dist):
        emp = self.empilhadeiras[emp_id]
        pos_atual = emp['posicao'] or ordem['origem']

        dist_total = matriz_dist.loc[pos_atual, ordem['origem']] + \
                     matriz_dist.loc[ordem['origem'], ordem['destino']]

        self.empilhadeiras[emp_id] = {
            'posicao': ordem['destino'],
            'livre_em': max(emp['livre_em'], ordem['data_hora']) + \
                        timedelta(seconds=dist_total/10),
            'distancia_total': emp['distancia_total'] + dist_total,
            'ordens_atendidas': emp['ordens_atendidas'] + [ordem.to_dict()]
        }

    def gerar_resultados(self, matriz_dist):
        resultados = []
        for emp_id, emp in self.empilhadeiras.items():
            for ordem in emp['ordens_atendidas']:
                resultados.append({
                    'ordem': ordem['ordem'],
                    'material': ordem['material'],
                    'origem': ordem['origem'],
                    'destino': ordem['destino'],
                    'empilhadeira': emp_id,
                    'distancia': matriz_dist.loc[ordem['origem'], ordem['destino']],
                    'inicio': ordem['data_hora']
                })

        metricas = {
            'total_ordens': len(resultados) + len(self.ordens_nao_atendidas),
            'atendidas': len(resultados),
            'nao_atendidas': len(self.ordens_nao_atendidas),
            'distancia_total': sum(e['distancia_total'] for e in self.empilhadeiras.values())
        }

        return pd.DataFrame(resultados), metricas

In [12]:
distancias = []

for i in range(1, 13):
  NUM_EMPILHADEIRAS = i

  # Processamento
  print("\nIniciando otimização...")
  otimizador = Otimizador(NUM_EMPILHADEIRAS)
  rotas, metricas = otimizador.otimizar(ordens, matriz_dist)

  distancias.append(metricas['distancia_total'])

  # Resultados
  print("\n=== RESUMO ===")
  print(f"Número de empilhadeiras: {NUM_EMPILHADEIRAS}")
  print(f"Ordens processadas: {metricas['total_ordens']}")
  print(f"Ordens atendidas: {metricas['atendidas']}")
  print(f"Ordens não atendidas: {metricas['nao_atendidas']}")
  print(f"Distância total: {metricas['distancia_total']:.2f}m")

distancias


Iniciando otimização...

=== RESUMO ===
Número de empilhadeiras: 1
Ordens processadas: 32412
Ordens atendidas: 32412
Ordens não atendidas: 0
Distância total: 9232718.66m

Iniciando otimização...

=== RESUMO ===
Número de empilhadeiras: 2
Ordens processadas: 32412
Ordens atendidas: 32412
Ordens não atendidas: 0
Distância total: 8903832.83m

Iniciando otimização...


KeyboardInterrupt: 

In [ ]:
# Salvar resultados
if not rotas.empty:
    rotas.to_excel('resultados_otimizacao.xlsx', index=False)
    files.download('resultados_otimizacao.xlsx')
    print("\nPlanilha com resultados disponibilizada para download!")
else:
    print("\nAVISO: Nenhuma rota foi gerada. Verifique os dados de entrada.")

print("\nPrimeiras linhas dos dados processados:")
print(rotas.head())